In [156]:
source("functions/data.R")
source("functions/process.R")
source("functions/simulations.R")

In [157]:
# Define dataframes
ETH <- get_data("ETH",params$start,params$end,params$period,52,FALSE)
XRP <- get_data("XRP",params$start,params$end,params$period,52,FALSE)
XMR <- get_data("XMR",params$start,params$end,params$period,52,FALSE)
LTC <- get_data("LTC",params$start,params$end,params$period,52,FALSE)

high <- cbind(ETH$high,XRP$high,XMR$high,LTC$high)
low <- cbind(ETH$low,XRP$low,XMR$low,LTC$low)
open <- cbind(ETH$open,XRP$open,XMR$open,LTC$open)
close <- cbind(ETH$close,XRP$close,XMR$close,LTC$close)

# Rename all the columns
colnames(high)  <- currency_vec
colnames(low) <- currency_vec
colnames(open) <- currency_vec
colnames(close) <- currency_vec
prices_vec <- list(high,low,open,close)

No encoding supplied: defaulting to UTF-8.
No encoding supplied: defaulting to UTF-8.
No encoding supplied: defaulting to UTF-8.
No encoding supplied: defaulting to UTF-8.


In [158]:
dim(ETH)

[1] 8440    7

In [46]:
# simulation function, the agent randomizes the weight vector at each step
# params: n_hist, number of timesteps in the past for price history
# params: n_episodes, number of episodes in the simulation
simulate_random <- function(n_hist,n_episodes){
    prev_v <- tail(head(close,n_hist-1),1) ## initialize first previous price vector
    returns <- 0

    # simulation
    for(i in 0:n_episodes){

        # generates random action (portfolio vector)
        random_action <- function(n_assets){
        x <- runif(n_assets+1)
        return(x/sum(x))
        }

        # get price dataframes based on current time steps
        hi <- head(high,i+n_hist)
        lo <- head(low,i+n_hist)
        price <- head(close,i+n_hist)

        wt <- random_action(4)
        curr_v <- tail(price,1)
        yt <- getPriceRelativeVec(prev_v,curr_v)
        rt <- getLogReturns(yt,wt)

        print(paste0("episode ",i))
        print(cat("wt: ",wt))
        print(cat("yt: ",yt))
        print(cat("rt: ",rt))
        print("=================================")

        prev_v <- curr_v # set previous price as current price
        returns <- update_returns(i+1,returns,yt,wt)
    }

    print(paste("average returns for this simulation: ", returns))
}

simulate_random(10,100)

[1] "episode 0"
wt:  0.09927672 0.1916044 0.3243538 0.1007339 0.2840312NULL
yt:  1 1.008287 1.007063 1.002357 0.9975541NULL
rt:  0.003415516NULL
[1] "================================="
[1] "episode 1"
wt:  0.09636395 0.3916544 0.2671937 0.01974814 0.2250399NULL
yt:  1 0.9733332 0.9667532 0.9824279 0.9903786NULL
rt:  -0.02208175NULL
[1] "================================="
[1] "episode 2"
wt:  0.3364182 0.02003909 0.2619481 0.1246081 0.2569865NULL
yt:  1 1.019516 1.01021 1.003953 1.014365NULL
rt:  0.007223558NULL
[1] "================================="
[1] "episode 3"
wt:  0.002945288 0.24775 0.3869249 0.3111157 0.05126411NULL
yt:  1 0.9599102 0.9571809 0.9760407 0.9683675NULL
rt:  -0.03622402NULL
[1] "================================="
[1] "episode 4"
wt:  0.2400109 0.1775872 0.1913318 0.1574327 0.2336375NULL
yt:  1 0.9734715 0.9888858 0.990435 0.9867474NULL
rt:  -0.01150569NULL
[1] "================================="
[1] "episode 5"
wt:  0.2284325 0.0007900176 0.1911992 0.2369632 0.342

In [33]:
# Function that gives actions table for n samples and n_curren currencies
get_actions_table <- function(n_samples,n_curren,state_vec){
    action_mat <- matrix(NA,nrow=n_samples,ncol=n_curren+1)
    for(i in 1:n_samples)
        action_mat[i,] <- random_action(n_curren)
    return(action_mat)
}

# Simulation function where the agent randomly samples actions at each step and observes next rewards
simulate_samples <- function(n_samples,n_episodes){
    price <- head(close,2) # initializes the price dataframe as the first 2 price vectors
    st <- getPriceRelativeVec(price[1,],price[2,]) # get first price relative vector (first state)
    prev_v <- tail(price,1) # initializes first price vector v
    training_mat <- matrix(nrow=1,ncol=11) # initialize training matrix for training data
    
    for(i in 0:n_episodes){
        # get price dataframe of current time step
        price <- head(close,i+1)
        
        # get the current v
        curr_v <- tail(price,1)
        
        # get price change
        yt <- getPriceRelativeVec(prev_v,curr_v)
        
        # sample actions
        action_mat <- get_actions_table(n_samples,4)
        
        # get rewards for each sampled action
        reward_vec <- c()
        for(j in 1:n_samples)
            reward_vec <- c(reward_vec,getLogReturns(yt,action_mat[j,]))
        
        # append rewards to matrix
        action_mat <- cbind(action_mat,reward_vec)
        
        # Add state columns to matrix
        for(val in rev(st))
            action_mat <- cbind(rep(val,n_samples),action_mat)
        
        # Concatenate matrix to training data
        training_mat <- rbind(training_mat,action_mat)
        
        print(paste0("episode ",i))
        print(action_mat)
        print(cat("st: ",st))
        print(cat("yt: ",yt))
        print("=================================")
        
        # update price vector
        prev_v <- curr_v
        
        # update state vector
        st <- yt
    }
    return(training_mat[-1,])
}

simulate_samples(10,10)

[1] "episode 0"
                                                                      
 [1,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.209418452 0.16600517
 [2,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.123708440 0.25510822
 [3,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.223056022 0.21702411
 [4,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.006938632 0.38705552
 [5,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.185096833 0.12303158
 [6,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.329482098 0.07960692
 [7,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.181111262 0.29145277
 [8,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.148944818 0.24346293
 [9,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.301588760 0.08882256
[10,] 1 0.9349467 0.9347246 0.9631573 0.9576903 0.064937472 0.33707658
                                       reward_vec
 [1,] 0.23555559 0.32354708 0.06547371 0.04235927
 [2,] 0.25434087 0.16348641 0.20335605 0.04950391
 [3,] 0.29851500 0.12464841 0.13675645 0.04569656
 [4

,,,,,,,,,,reward_vec
1,0.9349467,0.9347246,0.9631573,0.9576903,0.209418452,0.16600517,0.23555559,0.323547083,0.06547371,0.04235927
1,0.9349467,0.9347246,0.9631573,0.9576903,0.123708440,0.25510822,0.25434087,0.163486409,0.20335605,0.04950391
1,0.9349467,0.9347246,0.9631573,0.9576903,0.223056022,0.21702411,0.29851500,0.124648413,0.13675645,0.04569656
1,0.9349467,0.9347246,0.9631573,0.9576903,0.006938632,0.38705552,0.31780897,0.083006084,0.20519079,0.05955610
1,0.9349467,0.9347246,0.9631573,0.9576903,0.185096833,0.12303158,0.38282139,0.288008295,0.02104190,0.04615899
1,0.9349467,0.9347246,0.9631573,0.9576903,0.329482098,0.07960692,0.01625313,0.256793703,0.31786415,0.03008276
1,0.9349467,0.9347246,0.9631573,0.9576903,0.181111262,0.29145277,0.12288255,0.098107460,0.30644595,0.04511851
1,0.9349467,0.9347246,0.9631573,0.9576903,0.148944818,0.24346293,0.22150523,0.190261482,0.19582553,0.04720595
1,0.9349467,0.9347246,0.9631573,0.9576903,0.301588760,0.08882256,0.14201159,0.267776327,0.19980076,0.03456318
1,0.9349467,0.9347246,0.9631573,0.9576903,0.064937472,0.33707658,0.16671997,0.319279853,0.11198613,0.05093730


In [160]:
# simulates contextual bandits problem
# params
# n_curren: number of currencies in proble (including cash)
# n_steps: number of timesteps in the episode
# alpha: learning rate of the action preference functions
# window_size: size of rolling window of asset prices to consider in the action preferences
# discount: discount factor of previous price changes
simulate_contextual1 <- function(n_curren,n_steps,alpha,window_size,discount){
    
    Ht <- rep(0,5) # initialize preference vector
    
    # Initialize weight vector for moving average window
    weight_vec <- c()
    for(k in 0:(window_size-1))
        weight_vec <- c(weight_vec,discount^k)
    weight_vec <- rev(weight_vec)
    
    price <- head(close,window_size+1) # initializes the price dataframe as the first 2 price vectors
    #st <- getPriceRelativeVec(price[1,],price[2,]) # get first price relative vector (first state)
    
    # history vector of price changes
    history <- getPriceRelativeVec(price[1,],price[2,])
    for(h in 2:window_size){
        history <- rbind(history,getPriceRelativeVec(price[h,],price[h+1,]))
    }
    
    # Define state st as vector of discounted previous relative changes
    st <- weight_vec %*% history
    
    prev_v <- tail(price,1) # initializes first price vector v
    
    # Define reward vecs and random action reward vectors
    Rvec <- c()
    Rvec_random <- c()
    Rvec_market <- c()
    market_average <- rep(1/5,5)
    
    for(i in window_size:n_steps){
        # get price dataframe of current time step
        price <- head(close,i+2)
        # get the current v
        curr_v <- tail(price,1)
        # get price change
        yt <- getPriceRelativeVec(prev_v,curr_v)
        
        # update history matrix
        history <- rbind(tail(history,window_size-1),yt)
        
        # preference vector for state s (element-wise multiplication of Ht and st)
        Ht_s <- st * Ht
        
        # Compute pivec (softmaxes for each currency)
        piVec <- c()
        for(a in 1:n_curren)
            piVec <- c(piVec,get_softmax(a,Ht_s))
        
        # get the prefered action
        action <- which.max(piVec)
        
        # get the log returns for our action (in this case our action is the softax)
        rt <- exp(getLogReturns(yt,piVec))
        Rvec <- c(Rvec,rt)
        
        # Reward for random action
        Rvec_random <- c(Rvec_random,exp(getLogReturns(yt,random_action(4))))
        
        # Reward for market average
        Rvec_market <- c(Rvec_market,exp(getLogReturns(yt,market_average)))
        
        # Update preference vector
        Ht <- get_update(rt,Rvec,Ht,action,alpha)
        
        print(log(yt))
        print(Ht)
        print(piVec)
        print("=========================================")
        
#         print(paste0("episode",i))
#         print(cat("Ht: ", Ht))
#         print(cat("Ht_s: ", Ht_s))
        # print(cat("piVec: ", piVec))
#         print(cat("rt: ", rt))
        # print("==============================")
        
        prev_v <- curr_v
        st <- weight_vec %*% history
            
    }
    
    print(prod(Rvec))
    print(prod(Rvec_random))
    print(prod(Rvec_market))
    return(c(prod(Rvec),prod(Rvec_market)))
    
}

In [162]:
simulate_contextual1(5,4000,0.3,10,0.8)

[1] 0.000000000 0.019327864 0.010157799 0.003945475 0.014262934
[1] 0 0 0 0 0
[1] 0.2 0.2 0.2 0.2 0.2
[1] "========================================="
[1]  0.00000000 -0.04091556 -0.04376293 -0.02425101 -0.03214365
[1] -0.004477072  0.001119268  0.001119268  0.001119268  0.001119268
[1] 0.2 0.2 0.2 0.2 0.2
[1] "========================================="
[1]  0.000000000 -0.026886687 -0.011176423 -0.009611019 -0.013341178
[1] -0.0043526850  0.0012436545  0.0012436545  0.0012436545  0.0006224156
[1] 0.1960410 0.2009903 0.2009863 0.2009908 0.2009915
[1] "========================================="
[1] 0.00000000 0.03639174 0.03071206 0.01029014 0.01168835
[1] -5.625248e-03 -2.890862e-05 -2.890862e-05  6.326015e-03 -6.501475e-04
[1] 0.1961503 0.2010988 0.2010987 0.2011025 0.2005497
[1] "========================================="
[1]  0.000000000  0.006742651 -0.007109683  0.025717534  0.020452397
[1] -0.0062258965 -0.0006295569 -0.0006295569  0.0087096869 -0.0012507958
[1] 0.1950184 0.199950

[1] 2.589380 2.156528

In [163]:
# simulates contextual bandits problem
# This time action preferences are updated as if we pull all bandits and each timestep
# This is because we have full information of the environment
# params
# n_curren: number of currencies in proble (including cash)
# n_steps: number of timesteps in the episode
# alpha: learning rate of the action preference functions
# window_size: size of rolling window of asset prices to consider in the action preferences
# discount: discount factor of previous price changes
simulate_contextual2 <- function(n_curren,n_steps,alpha,window_size,discount){
    Ht <- rep(0,5) # initialize preference vector
    
    # Initialize weight vector for moving average window
    weight_vec <- c()
    for(k in 0:(window_size-1))
        weight_vec <- c(weight_vec,discount^k)
    weight_vec <- rev(weight_vec)
    
    price <- head(close,window_size+1) # initializes the price dataframe as the first 2 price vectors
    
    # history vector of price changes
    history <- getPriceRelativeVec(price[1,],price[2,])
    for(h in 2:window_size){
        history <- rbind(history,getPriceRelativeVec(price[h,],price[h+1,]))
    }
    
    # Define state st as vector of discounted previous relative changes
    st <- weight_vec %*% history
    
    prev_v <- tail(price,1) # initializes first price vector v
    
    # Define reward vecs and random action reward vectors
    Rvec <- c()
    Rvec_random <- c()
    Rvec_market <- c()
    market_average <- rep(1/5,5)
    Rmat <- matrix(0,nrow=1,ncol=n_curren)
    
    for(i in window_size:n_steps){
        # get price dataframe of current time step
        price <- head(close,i+2)
        # get the current v
        curr_v <- tail(price,1)
        # get price change
        yt <- getPriceRelativeVec(prev_v,curr_v)
        
        # update history matrix
        history <- rbind(tail(history,window_size-1),yt)
        
        # preference vector for state s (element-wise multiplication of Ht and st)
        Ht_s <- st * Ht
        
        # Compute pivec (softmaxes for each currency)
        piVec <- c()
        for(a in 1:n_curren)
            piVec <- c(piVec,get_softmax(a,Ht_s))
        
        # get the log returns for our action (in this case our action is the softax)
        rt <- exp(getLogReturns(yt,piVec))
        Rvec <- c(Rvec,rt)
        
        # get the log returns for pulling each arm
        Rvec_bandits <- c()
        for(a in 1:n_curren)
            Rvec_bandits <- c(Rvec_bandits,log(yt[a]))
        
        # update preference vector
        Ht <- get_update_all_bandits(Rvec_bandits,Rmat,Ht,alpha)
        print(Rvec_bandits)
        print(Ht)
        print(piVec)
        print("============================================")
        
        # check if we are at first timestep, and if so, initialize Rmat, else append to Rmat
        if(i == window_size)
            Rmat <-  matrix(Rvec_bandits,nrow=1,ncol=n_curren)
        else
            Rmat <- rbind(Rmat,Rvec_bandits)
        
        # Reward for random action
        Rvec_random <- c(Rvec_random,exp(getLogReturns(yt,random_action(4))))
        
        # Reward for market average
        Rvec_market <- c(Rvec_market,exp(getLogReturns(yt,market_average)))
        
        # update prevsous price and state
        prev_v <- curr_v
        st <- weight_vec %*% history
        
    }
    print(prod(Rvec))
    print(prod(Rvec_random))
    print(prod(Rvec_market))
    return(c(prod(Rvec),prod(Rvec_market)))
    
}


In [164]:
simulate_contextual2(5,4000,0.6,10,0.8)

[1] 0.000000000 0.019327864 0.010157799 0.003945475 0.014262934
[1] 0.000000000 0.009277375 0.004878012 0.001895170 0.006851704
[1] 0.2 0.2 0.2 0.2 0.2
[1] "============================================"
[1]  0.00000000 -0.04091556 -0.04376293 -0.02425101 -0.03214365
[1]  0.00000000 -0.01960548 -0.02096481 -0.01161131 -0.01533444
[1] 0.1959369 0.2042238 0.2002311 0.1975951 0.2020131
[1] "============================================"
[1]  0.000000000 -0.026886687 -0.011176423 -0.009611019 -0.013341178
[1]  0.00000000 -0.02734184 -0.01826025 -0.01135146 -0.01744732
[1] 0.2121813 0.1945547 0.1934617 0.2015528 0.1982495
[1] "============================================"
[1] 0.00000000 0.03639174 0.03071206 0.01029014 0.01168835
[1]  0.000000000 -0.002039555  0.003692773 -0.001611106 -0.006804565
[1] 0.2134196 0.1891622 0.1968976 0.2029579 0.1975627
[1] "============================================"
[1]  0.000000000  0.006742651 -0.007109683  0.025717534  0.020452397
[1] 0.000000000 0.002647

[1] 1.890511 2.156528

In [64]:
library(nnet)

for(i in 1:10){
    Var1 <- runif(50,0,100)
    sqrt.data <- data.frame(Var1,Sqrt=sqrt(i) + Var1)
    
    if(i == 1){
        net.sqrt <- nnet(Sqrt~Var1, sqrt.data, size=10, maxit=1)
    } else {
        net.sqrt <- nnet(Sqrt~Var1,  sqrt.data, size=10, maxit=10, Wts=net.sqrt$wts)
    }
        
    
    print(net.sqrt$wts)
    print("==========================================")
}

# # Make Some Training Data
# Var1 <- runif(50, 0, 100) 
# # create a vector of 50 random values, min 0, max 100, uniformly distributed
# sqrt.data <- data.frame(Var1, Sqrt=sqrt(Var1)) 
# # create a dataframe with two columns, with Var1 as the first column
# # and square root of Var1 as the second column

# nnet.sqrt <- nnet(Sqrt~Var1, sqrt.data, size=10, maxit=1)

# weights:  31
initial  value 188833.730608 
final  value 188233.080057 
stopped after 2 iterations
 [1]    2.88789116154  148.79646982064    0.45214161866   -0.11089756633
 [5]   -0.56274653382   -0.63095494494    8.47060269185  579.74885503230
 [9]   -2.46766349706 -140.32764511513    0.25362085619    0.62571131094
[13]   -0.62319842786    0.79215028720    0.13087912925    0.70790294812
[17]    0.07373739906    0.20756312574    0.44637704047   -2.06535431783
[21]  107.57813095754   95.33067830604    0.73723544218    0.02995010330
[25]   37.80351465087   41.19112383881  107.46186139834  107.48819140182
[29]  107.28689762848  106.75602321898   -0.29050323268
[1] "=========================================="
# weights:  31
initial  value 155365.654136 
final  value 155365.654136 
converged
 [1]    2.88789116154  148.79646982064    0.45214161866   -0.11089756633
 [5]   -0.56274653382   -0.63095494494    8.47060269185  579.74885503230
 [9]   -2.46766349706 -140.32764511513    0.25362085619

In [55]:
nnet.sqrt <- nnet(Sqrt~Var1, sqrt.data, size=10, maxit=1)

# weights:  31
initial  value 13801.935881 
final  value 13247.836022 
stopped after 2 iterations


In [14]:
# Try a new method based on sampling actions and approximating action value function

# function used to sample a random action
sample_action <- function(n_assets){
    x <- runif(n_assets+1)
    return(x/sum(x))
}

# Make Some Training Data
Var1 <- runif(50, 0, 100) 
# create a vector of 50 random values, min 0, max 100, uniformly distributed
sqrt.data <- data.frame(Var1, Sqrt=sqrt(Var1)) 
# create a dataframe with two columns, with Var1 as the first column
# and square root of Var1 as the second column

# Train the neural net
net.sqrt <- neuralnet(Sqrt~Var1,  sqrt.data, hidden=10, threshold=0.01)
# train a neural net, try and predict the Sqrt values based on Var1 values
# 10 hidden nodes

# Compute or predict for test data, (1:10)^2
compute(net.sqrt, (1:10)^2)$net.result
# What the above is doing is using the neural net trained (net.sqrt), 
# if we have a vector of 1^2, 2^2, 3^2 ... 10 ^2 (i.e. 1, 4, 9, 16, 25 ... 100), 
# what would net.sqrt produce?

1.254774474
1.997149954
2.994169568
4.001415595
4.997892565
6.002643974
6.999658074
7.998341659
9.003371897
9.972014956


In [43]:
# get price relative vector for starting state
price <- head(close,2)
yt <- getPriceRelativeVec(head(price,1),tail(price,1))
prev_v <- tail(price,1)
returns <- 0
data <- c()

for(i in 1:100){
    
    price <- head(close,i+2) # update price matrix 
    curr_v <- tail(price,1)
    yt_next <- getPriceRelativeVec(prev_v,curr_v) # get price relative vector of next state
    wt <- sample_action(4) # sample an action
    rt <- getLogReturns(yt_next,wt)
    
    data <- rbind(data,c(yt,wt,rt)) # append state action pair and reward to data
    
    prev_v <- curr_v
    yt <- yt_next
}

In [44]:
# create a testing dataframe
data <- data.frame(data)
data$X1 <- NULL
colnames(data) <- c("y2","y3","y4","y5","w1","w2","w3","w4","w5","r")

In [45]:
# fit neural net and predict
network <- neuralnet(r~y2+y3+y4+y5+w1+w2+w3+w4+w5,head(data,50),hidden=10,threshold=0.01)
cbind(compute(network,subset(tail(data,50),select=-r))$net.result,tail(data,50)$r)

51,-0.0079558480769,0.0708140701148
52,0.0245109623899,0.0672985527889
53,-0.0352382795099,-0.0049296967756
54,0.0147700499662,-0.0391479558467
55,0.0047419403723,0.0128816615937
56,0.0257578968284,-0.0069060369898
57,0.0173938037127,-0.0362066941949
58,0.0185186165525,-0.0327041260752
59,-0.0297483537749,0.0225316904661
60,0.0115562279471,-0.0168263977611
61,0.0055204617392,0.0022738971048


In [144]:
alpha=0.6    #initializing the different probabilities and reward 
beta=0.8
rsearch=2
rwait=1
DF=0.9
State=c(1, 2)
Action=c(1,2,3)
s=c(rep(State[1],2),rep(State[2],2),rep(State[1],2),rep(State[2],4),1) #initializing the matrix
a=c(rep(Action[1],4),rep(Action[2],4),rep(Action[3],3))
sprime=c(rep(State[1:2],5),1)
r=c(rep(rsearch,2),-3, rsearch,rwait,0,0,rwait,0,0,0)
transprob=c(alpha,(1-alpha),(1-beta),beta,1,0,0,1,1,0,1)
SASRp=matrix(c(s,a,sprime,r,transprob),ncol=5,nrow=11,byrow=FALSE)
colnames(SASRp) = c("s","a","s'","r","p(s',r|s,a)")
SASRp
i=1  #removing rows where the transition prob is 0
j=11
while( i<=j){   
  if (SASRp[i,5]==0){
    SASRp=SASRp[-i,]
    j=j-1
    i=i-1
    
  }
  i=i+1
}

CalculatePolicyStateValueFunction=function(Policy,SASRp,DF){
  #to make it easier we will append the Pi(a|s) to SASRp
  SASRpMOD=cbind(SASRp[,1:5],c(rep(0,8)))
  colnames(SASRpMOD) = c("s","a","s'","r","p(s',r|s,a)","pi(a|s)")
  for(i in 1:8){
  for(a in 1:3){
    for(s in 1:2){
      if(SASRp[i,2]==Action[a]&SASRp[i,1]==State[s]){
        SASRpMOD[i,6]=Policy[s,a]
      }
        
    }
  }
  }
 
  b=c(0,0)
  M=matrix(data=0,nrow=2,ncol=2)
  #calculation for b
  for(i in 1:2){
    for(j in 1:3){
      temp.index=which(SASRp[,"a"]==j &SASRp[,"s"]==i)
      for (t  in temp.index) {
        b[i]=b[i]+SASRp[t,"r"]*SASRp[t,5]*SASRpMOD[t,6]
      }
    }
  }
  b
  
  #calculation for M
  for(i in 1:2){
    for(j in 1 :2){
      for(l in 1:3){
        temp.index=which(SASRpMOD[,3]==j & SASRp[,"s"]==i & SASRp[,"a"]==l)
        for( t in temp.index){
          M[i,j]=M[i,j]+DF*SASRp[t,5]*SASRpMOD[t,6]
        }
      }
    }
  }
  
  v=solve(diag(2)-M)%*%b
  as.vector(v)
  return(v)
  VFALLPolicies

}
CalculatePolicyStateValueFunction(Policy,SASRp,DF)


CalculatePolicyActionValueFunction=function(Policy,SASRp,DF){
  v=CalculatePolicyStateValueFunction(Policy,SASRp,DF)
  PolicyAVF=matrix(data=0,nrow=2,ncol=3)
  for(i in 1:2){
    for(j in 1:3){
      for(z in 1:2){
        temp.index=which(SASRp[,"a"]==j &SASRp[,"s"]==i & SASRp[,3]==z)
        for(t in temp.index){
          PolicyAVF[i,j]=PolicyAVF[i,j]+SASRp[t,5]*(SASRp[t,"r"]+DF*v[z])
        }
      }
    }  
    
  }
  return(PolicyAVF) 
}

PolicyAVF=CalculatePolicyActionValueFunction(Policy,SASRp,DF)
PolicyAVF


######Part b####
initstate=1
Eplength=10
State.Vector=c(initstate,rep(0,Eplength))
EpReward=c()
theseed=100
Policy=matrix(c(0.7,0.2,0.1,0.1,0.7,0.2),nrow = 2,ncol=3,byrow = T)
EpisodeOutcomes=matrix(data = 0,nrow=3,ncol=Eplength)
rownames(EpisodeOutcomes)=c("S","A","R")
set.seed(theseed)
for( i in 1:Eplength){
  temp.action.prob=Policy[State.Vector[i],]
  action=sample(Action,size = 1,prob =temp.action.prob,replace = T )
  temp.index=which(SASRp[,"s"]==State.Vector[i]&SASRp[,"a"]==action)
  if(length(SASRp[temp.index,"sprime"])==1){
    State.Vector[i+1]=SASRp[temp.index,"sprime"]
  }else{
    State.Vector[i+1]=sample(SASRp[temp.index,"sprime"],size = 1,prob = SASRp[temp.index,"transprob"])
  }
  temp.index2=which(SASRp[,"s"]==State.Vector[i]&SASRp[,"a"]==action & SASRp[,"sprime"]==State.Vector[i+1])
  EpReward[i]=SASRp[temp.index2,"r"]
  EpisodeOutcomes[,i]=c(State.Vector[i+1],action,EpReward[i])
  
  
}








#### Question 2 ####
#### Part A ####
a=c(-1,1)

thetavec=c(-1,1)
States=c(1:4)
reward=c()
averagereward=c()
Gt=c()

SimulCorridorEpisodeFixedPolicy=function(pright, theSeed){ #where pright is Pi(1,thetavec), since there is only 2 states Pi(-1,tethavec)=1-pright
  set.seed(theSeed)
  i=1
  actionseq=c()
  stateseq=c()
  stateseq[1]=1
  repeat{
    if(stateseq[i]==4){
      actionseq[i]= NaN
      break}
    u=runif(1)
    if(u<=pright){
      actionseq[i]=1
      if(stateseq[i]==2){
        stateseq[i+1]=stateseq[i]-1
      } else{
        stateseq[i+1]=stateseq[i]+1
      }
    } else{
      actionseq[i]=-1
      if(stateseq[i]==2){
        stateseq[i+1]=stateseq[i]+1
      } else if(stateseq[i]==1) {
        stateseq[i+1]=stateseq[i]
      } else{
        stateseq[i+1]=stateseq[i]-1
      }
      
    }
    
    
    i=i+1
  }
  mat=matrix(c(stateseq,actionseq),byrow = F,ncol = 2, nrow = length(stateseq))
  colnames(mat)=c("Sequence of visited states","Sequence of actions taken")
  reward=-length(actionseq)
  averagereward=mean(reward)
  
  return((mat))
}
nrow(SimulCorridorEpisodeFixedPolicy(0.05,2))

reward0.95=c()
for(i in 1:2000){
reward0.95[i]=-nrow(SimulCorridorEpisodeFixedPolicy(0.95,i))
}
mean(reward0.95)
reward0.05=c()
for(i in 1:2000){
  reward0.05[i]=-nrow(SimulCorridorEpisodeFixedPolicy(0.05,i))
}
mean(reward0.05)

OneRunCorridor=function(inittheta,alphasteptheta,nepis, theseed){
  RewardEachEpisode=c()
  ProbRightEachEpisode=c()
  xsa=matrix(c(ifelse(a==-1,1,0),ifelse(a==1,1,0)),ncol = 2, byrow = T)
  colnames(xsa)=c("a=-1","a=1")
  row.names(xsa)=c("left","right")
  for(i in 1:nepis){
    hsatheta=inittheta%*%xsa
    piaTheta=(exp(hsatheta)/sum(exp(hsatheta)))
    RewardEachEpisode[i]=-nrow(SimulCorridorEpisodeFixedPolicy(piaTheta[1],theseed+i))
    ProbRightEachEpisode[i]=piaTheta[1]
    gradlogpi=c(ifelse(a==1,1-piaTheta[1],1-piaTheta[2]))
    inittheta=inittheta+alphasteptheta*RewardEachEpisode[i]*gradlogpi
  }
 return(list("Reward"=RewardEachEpisode,"Probability of choosing right"=ProbRightEachEpisode))
  
}

OneRunCorridor(inittheta = c(log((1/0.95-1)),0), alphasteptheta = 2^-12,nepis = 1000,1980)
alpha=c(2^-12,2^-13,2^-14)
resultmat=array(NA,dim=c(100,1000,3))
probrightmat=array(NA,dim=c(100,1000,3))
for(alphasize in 1:3){
  for(run in 1:100){
    result1run=OneRunCorridor(inittheta = c(log((1/0.95-1)),0), alphasteptheta = alpha[alphasize],nepis = 1000,1980+run)
    for(episode in 1:1000){
    resultmat[run,episode,alphasize]=result1run[[1]][episode]
    probrightmat[run,episode,alphasize]=result1run[[2]][episode]
    }
    
  }
  
  
  
}
#Calculating the average reward on episode per 100 run
MeanonEpisode=matrix(NA, ncol = 3, byrow = F, nrow = 1000)
for(j in 1:3){
for(i in 1:1000){
MeanonEpisode[i,j]=mean(resultmat[,i,j])
}
}
plot(x =1:1000,y=MeanonEpisode[,1],xlab = "Episode", ylab = "Total reward on Episode (Average over 100 run)",col="blue")
lines(x=1:1000,y = MeanonEpisode[,2], col="red")
lines(x=1:1000, y = MeanonEpisode[,3],col="green")
abline(h=-11.6, col="gray")
legend("bottomright", c("alpha=2^-12","alpha=2^-13","alpha=2^-14","v*(s0)"), fill=c("blue", "red","green","gray"),cex = 0.6)
piaTheta=0.999

s,a,s',r,"p(s',r|s,a)"
1,1,1,2,0.6
1,1,2,2,0.4
2,1,1,-3,0.2
2,1,2,2,0.8
1,2,1,1,1.0
1,2,2,0,0.0
2,2,1,0,0.0
2,2,2,1,1.0
2,3,1,0,1.0
2,3,2,0,0.0


ERROR: Error in CalculatePolicyStateValueFunction(Policy, SASRp, DF): object 'Policy' not found


In [155]:
a=c(-1,1)

thetavec=c(-1,1)
States=c(1:4)
reward=c()
averagereward=c()
Gt=c()

SimulCorridorEpisodeFixedPolicy=function(pright, theSeed){ #where pright is Pi(1,thetavec), since there is only 2 states Pi(-1,tethavec)=1-pright
  set.seed(theSeed)
  i=1
  actionseq=c()
  stateseq=c()
  stateseq[1]=1
  repeat{
    if(stateseq[i]==4){
      actionseq[i]= NaN
      break}
    u=runif(1)
    if(u<=pright){
      actionseq[i]=1
      if(stateseq[i]==2){
        stateseq[i+1]=stateseq[i]-1
      } else{
        stateseq[i+1]=stateseq[i]+1
      }
    } else{
      actionseq[i]=-1
      if(stateseq[i]==2){
        stateseq[i+1]=stateseq[i]+1
      } else if(stateseq[i]==1) {
        stateseq[i+1]=stateseq[i]
      } else{
        stateseq[i+1]=stateseq[i]-1
      }
      
    }
    
    
    i=i+1
  }
  mat=matrix(c(stateseq,actionseq),byrow = F,ncol = 2, nrow = length(stateseq))
  colnames(mat)=c("Sequence of visited states","Sequence of actions taken")
  reward=-length(actionseq)
  averagereward=mean(reward)
  
  return((mat))
}

OneRunCorridorBaseLine=function(inittheta,initw,alphasteptheta,alphastepw,nepis,theseed){
  RewardEachEpisode=c()
  ProbRightEachEpisode=c()
  xsa=matrix(c(ifelse(a==-1,1,0),ifelse(a==1,1,0)),ncol = 2,nrow=2, byrow = T)
  colnames(xsa)=c("a=-1","a=1")
  row.names(xsa)=c("left","right")
  
  for(i in 1:nepis){
    hsatheta=inittheta%*%xsa
    piaTheta=(exp(hsatheta)/sum(exp(hsatheta)))
    if(piaTheta[1]>=0.99){
      piaTheta[1]=0.99
    } else{
      piaTheta[1]=piaTheta[1]}
    if(piaTheta[1]<=0.01){
      piaTheta[1]=0.01
    }else{ 
      piaTheta[1]=piaTheta[1]}
    RewardEachEpisode[i]=-nrow(SimulCorridorEpisodeFixedPolicy(piaTheta[1],theseed+i))
    ProbRightEachEpisode[i]=piaTheta[1]
    gradlogpi=c(ifelse(a==1,1-piaTheta[1],1-piaTheta[2]))
    
    for(s in 1:(nrow(SimulCorridorEpisodeFixedPolicy(piaTheta[1],theseed+i))-1)){
    delta=1
      if(SimulCorridorEpisodeFixedPolicy(piaTheta[1],theseed+i)[s,2]==1){
      initw[1]=initw[1]+alphastepw*(RewardEachEpisode[i]-initw[1])
      delta=RewardEachEpisode[i]-initw[1]
      } else{
        initw[2]=initw[2]+alphastepw*(RewardEachEpisode[i]-initw[2])
      delta=RewardEachEpisode[i]-initw[2]
        }
    }
    inittheta=inittheta+delta*alphasteptheta*RewardEachEpisode[i]*gradlogpi
    print(delta)
    print(alphasteptheta)
      print(RewardEachEpisode[i])
      print(gradlogpi)
      print("=======================================")
    }
  return(list("Reward"=RewardEachEpisode,"Probability of choosing right"=ProbRightEachEpisode))
  
}




OneRunCorridorBaseLine(inittheta = c(log((1/0.999-1)),0),initw = c(0,0,0,0), alphasteptheta = 2^-9,alphastepw = 2^-12,nepis = 100,theseed = 12)

SimulCorridorEpisodeFixedPolicy(0.95,10)[,2]
c=1
ifelse(c>1,c=2,c=3)                                         


[1] -176.9136
[1] 0.001953125
[1] -177
[1] 0.001 0.990
[1] "======================================="
[1] -98.86529
[1] 0.001953125
[1] -99
[1] 0.00 0.99
[1] "======================================="
[1] -217.7589
[1] 0.001953125
[1] -218
[1] 0.00 0.99
[1] "======================================="
[1] -268.6277
[1] 0.001953125
[1] -269
[1] 0.00 0.99
[1] "======================================="
[1] -325.151
[1] 0.001953125
[1] -326
[1] 0.00 0.99
[1] "======================================="
[1] -562.3266
[1] 0.001953125
[1] -564
[1] 0.00 0.99
[1] "======================================="
[1] -591.7484
[1] 0.001953125
[1] -594
[1]  NaN 0.99
[1] "======================================="


ERROR: Error in if (piaTheta[1] >= 0.99) {: missing value where TRUE/FALSE needed


In [140]:
OneRunCorridorBaseLine(inittheta = c(log((1/0.999-1)),0),initw = c(0,0,0,0), alphasteptheta = 2^-9,alphastepw = 2^-12,nepis = 100,theseed = 12)

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [1] not equal to array extent
